In [1]:
import cantera as ct
import numpy as np
import pandas as pd

# Biofuel Analysis

## Part A - Fuel Properties

In [2]:
#proximate analysis. %mass
proximate_analysis = {'moisture' : 20,'ash' : 1,'volitile_matter' : 70.5 ,'fixed_carbon' : 8.5}
#ultimate analysis . %mass
ultimate_analysis = {'C':55, 'H':11, 'O':33, 'N':1, 'S':0}

#use a dictionary to map elements to their weights
weights = {'H' : 1.01, 'C' : 12.01, 'O' : 16.00, 'N' : 14.01, 'S' : 32.06, 'H2O':18.02}

In [3]:
db = pd.DataFrame([ultimate_analysis, weights],index=['UA','KG'])
db

,C,H,H2O,N,O,S
UA,55.00,11.00,NaN,1.00,33.0,0.00
KG,12.01,1.01,18.02,14.01,16.0,32.06


In [4]:
#check sums to 100%
sum(proximate_analysis.values())

100.0

In [5]:
#check sums to 100%
sum(proximate_analysis.values())

100.0

In [6]:
#find DAF per kg fuel
per_kg = (proximate_analysis['volitile_matter']+proximate_analysis['fixed_carbon']) / sum(proximate_analysis.values())
per_kg

0.79

In [7]:
mole_H = per_kg*ultimate_analysis['H']/weights['H'] 
mole_C = per_kg*ultimate_analysis['C']/weights['C'] 
mole_O = per_kg*ultimate_analysis['O']/weights['O'] 
mole_N = per_kg*ultimate_analysis['N']/weights['N']
mole_S = per_kg*ultimate_analysis['S']/weights['S']
mole_H2O = proximate_analysis['moisture']/weights['H2O']

kmolsPer100Kg = {'C': mole_C, 'H':mole_H, 'O':mole_O, 'N':mole_N, 'H2O':mole_H2O}
kmolsPer100Kg

{'C': 3.61781848459617,
 'H': 8.603960396039605,
 'H2O': 1.1098779134295227,
 'N': 0.05638829407566025,
 'O': 1.629375}

In [8]:
fuel = ct.Solution('gri30.cti')
fuel.TP = 298, 101325
fuel.X = kmolsPer100Kg

In [9]:
#Now we create a solution object'gas'. We set this object to react with air a phi of 1.15
gas_react = ct.Solution('gri30.cti')
gas_react.TP = 600, 101325
gas_react.set_equivalence_ratio(1/1.15, kmolsPer100Kg, 'O2:1.0, N2:3.76')

#find ratio for 100kg of fuel
moles_react = gas_react.mole_fraction_dict()
ratio = kmolsPer100Kg['H']/moles_react['H']

In [10]:
#put in terms of 100kg of fuel
for key in moles_react:
    moles_react[key] *= ratio
    
moles_react

{'C': 3.6178184845961705,
 'H': 8.603960396039605,
 'H2O': 1.1098779134295227,
 'N': 0.05638829407566026,
 'N2': 21.421619565512653,
 'O': 1.629375,
 'O2': 5.697239246146982}

### Find the Enthalpy of Formation using the LHV

In [11]:
#We can check our had calc
h_react = gas_react.h/1000
h_react

5645.021126646164

### Combustion Check

In [12]:
#Here we combust the reactants at a temperature and pressure
gas_react.equilibrate('TP')

In [13]:
#Becayse gri30 has lots of minor species we will grab only the molecules we are interested in for this problem
molefract = gas_react.mole_fraction_dict()
smpMolFract = {'CO2':molefract['CO2'], 'H2O':molefract['H2O'] , 'N2':molefract['N2'] , 'O2':molefract['O2']}
smpMolFract

{'CO2': 0.11587175716953525,
 'H2O': 0.1733313906225026,
 'N2': 0.6869962056312481,
 'O2': 0.023800627538585584}

In [14]:
#Now we find the ratio that will convert us to 100kg of raw fuel
ratio = moles_react['C']/smpMolFract['CO2']

In [15]:
#Let's loop through the products and multiply by the ratio
for key in smpMolFract:
    smpMolFract[key] *= ratio

#Now we print the dictionary that shows the moles of each product at 100kg fuel
smpMolFract

{'CO2': 3.6178184845961705,
 'H2O': 5.4118581116997175,
 'N2': 21.44981341694561,
 'O2': 0.7431176704095281}

## Part B -  Combustion Analysis

In [16]:
'''
Let's analyze the reactants. What are their enthalpies at different temperatures
This code block finds the enthalpies of oxygen and nitrogen at both 298K and 500K
'''

gas = ct.Solution('gri30.xml')
gas.basis = 'molar'

# Stream A1 (O2)
A1 = ct.Quantity(gas, constant='HP')
A1.TPX = 298, ct.one_atm, 'O2:1'

# Stream B1 (N2)
B1 = ct.Quantity(gas, constant='HP')
B1.TPX = 298, ct.one_atm, 'N2:3.76'

# Stream C1 (H2O)
C1 = ct.Quantity(gas, constant='HP')
C1.TPX = 298, ct.one_atm, 'H2O:1'

# Stream A (O2)
A = ct.Quantity(gas, constant='HP')
A.TPX = 500, ct.one_atm, 'O2:1'

# Stream B (N2)
B = ct.Quantity(gas, constant='HP')
B.TPX = 500, ct.one_atm, 'N2:3.76'

# Stream C (H2O)
C = ct.Quantity(gas, constant='HP')
C.TPX = 500, ct.one_atm, 'H2O:1'


In [17]:
#calculate h-h298

#oxygen
hO2 = (A.h - A1.h)/1000
print(hO2)

#nitrogen
hN2 = (B.h - B1.h)/1000
print(hN2)

#steam
hH2O = (C.h - C1.h)/1000
print(hH2O)

6090.594296961835
5924.513366646463
6929.860983924419


In [27]:
'''
Let's analyze the products. What are their enthalpies at different temperatures
This code block finds the enthalpies of H2O, CO2, N2, and O2 at both 298K and 600K
'''

gas = ct.Solution('gri30.xml')
gas.basis = 'molar'

# Stream A1 (H2O)
A1 = ct.Quantity(gas, constant='HP')
A1.TPX = 298, ct.one_atm, 'H2O:1'

# Stream B1 (CO)
B1 = ct.Quantity(gas, constant='HP')
B1.TPX = 298, ct.one_atm, 'CO:1'

# Stream C1 (H2)
C1 = ct.Quantity(gas, constant='HP')
C1.TPX = 298, ct.one_atm, 'H2:1'

# Stream D1 (N2)
D1 = ct.Quantity(gas, constant='HP')
D1.TPX = 298, ct.one_atm, 'N2:1'


# Stream A (H2O)
A = ct.Quantity(gas, constant='HP')
A.TPX = 500, ct.one_atm, 'H2O:1'

# Stream B (CO)
B = ct.Quantity(gas, constant='HP')
B.TPX = 500, ct.one_atm, 'CO:1'

# Stream C1 (H2)
C = ct.Quantity(gas, constant='HP')
C.TPX = 500, ct.one_atm, 'H2:1'

# Stream D1 (N2)
D = ct.Quantity(gas, constant='HP')
D.TPX = 448, ct.one_atm, 'N2:1'

In [28]:
#calculate h-h298

#oxygen
hH2O = (A.h - A1.h)/1000
print(hH2O)

#nitrogen
hCO2 = (B.h - B1.h)/1000
print(hCO2)

#oxygen
hN2 = (C.h - C1.h)/1000
print(hN2)

#nitrogen
hO2 = (D.h - D1.h)/1000
print(hO2)

6929.860983924419
5940.071469085693
5899.790438394021
4388.23669376562


# Part B

### Biomass Fed through Gasification

### 3 - Oxygen Separator

In [20]:
#Create solution for Air, O2, and N2
air = ct.Solution('gri30.cti')
air.basis = 'molar'
air.X = {'O2':1, 'N2':3.76}
air.TP = 298, 101325

Nitrogen = ct.Solution('gri30.cti')
Nitrogen.basis = 'molar'
Nitrogen.X = {'N2':1}
Nitrogen.TP = 298, 101325

Oxygen = ct.Solution('gri30.cti')
Oxygen.basis = 'molar'
Oxygen.X = {'O2':1}
Oxygen.TP = 298, 101325

In [21]:
#First let's find the energy required to separate O2 and N2 and a per kmol O2 basis

To = 298
so = 4.76*air.s
s1 = (3.76*Nitrogen.s + Oxygen.s)

mol_w = (To*(s1-so))/1000
mol_w

-6062.865513509683

In [22]:
#Now let's convert that to kg fuel
molMassO2 = 32
mass_w = mol_w / molMassO2
massFuel_w = mass_w*0.179
massFuel_w

-33.91415396619479